In [1]:
import pandas as pd
import numpy as np

# Load the data
try:
    # For lead time history
    df_master = pd.read_csv('../data/master_table.csv')

    # For demand predictions
    df_forecast = pd.read_csv('../data/forecast_output.csv')

    print('Data loaded sucessfully!')
except FileNotFoundError:
    print('File not found!')

# Hero product
top_product = df_master.groupby('product_id')['price'].sum().sort_values(ascending=False).index[0]
print(f"Analyzing prosuct: {top_product}")


Data loaded sucessfully!
Analyzing prosuct: bb50f2e236e5eea0100680137654686c


In [2]:
# Lead time analysis

# Filter out products
df_product_history = df_master[df_master['product_id'] == top_product].copy()

# Calculate average lead time
avg_lead_time_days = df_product_history['actual_lead_time_days'].mean()

# Calculate lead time variability (L_std) - This is risk!
std_lead_time_days = df_product_history['actual_lead_time_days'].std()

# Convert to weeks
avg_lead_time_weeks = avg_lead_time_days / 7
std_lead_time_weeks = std_lead_time_days / 7

print(f"Average Lead Time: {avg_lead_time_days:.1f} days ({avg_lead_time_weeks:.1f} weeks)")
print(f"Lead Time Risk (Std Dev): {std_lead_time_days:.1f} days")

Average Lead Time: 12.9 days (1.8 weeks)
Lead Time Risk (Std Dev): 11.2 days


In [3]:
# Demand analysis

next_12_weeks = df_forecast.tail(12).copy()

# Average expected demand per week (D_avg)
avg_demand_weekly = next_12_weeks['yhat'].mean()

# Demand variability (D_std)
next_12_weeks['uncertainty'] = next_12_weeks['yhat_upper'] - next_12_weeks['yhat_lower']
std_demand_weekly = next_12_weeks['uncertainty'].mean() / 4

print(f"Forecast Weekly Demand: {avg_demand_weekly:.0f} units")
print(f"Demand Risk (Std Dev): {std_demand_weekly:.0f} units")

Forecast Weekly Demand: 6 units
Demand Risk (Std Dev): 2 units


In [4]:
# Calculate safety stock and reorder point

# Service Level (95% = Z-score of 1.65)
Z_score = 1.65

# Formula
# Part 1: Variance due to demand fluctuations
term1 = avg_lead_time_weeks * (std_demand_weekly ** 2)

# Part 2: Variance due to shipping delays (Lead Time)
term2 = (avg_demand_weekly ** 2) * (std_lead_time_weeks ** 2)

safety_stock = Z_score * np.sqrt(term1 + term2)

# Calculate the Reorder Point (ROP)
# ROP = (Lead Time * Demand) + Safety Stock
reorder_point = (avg_lead_time_weeks * avg_demand_weekly) + safety_stock

# Recommendation
print("--- INVENTORY OPTIMIZATION REPORT ---")
print(f"Product ID: {top_product}")
print(f"Safe Level (Safety Stock): {safety_stock:.0f} units")
print(f"Trigger to Buy (Reorder Point): {reorder_point:.0f} units")
print("-" * 30)
print(f"Rec: When inventory drops below {reorder_point:.0f}, place an order immediately.")

--- INVENTORY OPTIMIZATION REPORT ---
Product ID: bb50f2e236e5eea0100680137654686c
Safe Level (Safety Stock): 18 units
Trigger to Buy (Reorder Point): 30 units
------------------------------
Rec: When inventory drops below 30, place an order immediately.
